<a href="https://colab.research.google.com/github/CannibalKush/hack-week-ml-classifier/blob/main/Yoco_hack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Welcome to team ML Risk\'s hack week project!

We'll be using machine learning and building a simple classifier to sort data, in the form of a merchant's transactions and history, into categories of "credit-worthiness".

To do so, we plan on using a real-life Yoco dataset and Google's Tensorflow. Hopefully at the end of the day we can start thinking about how to implement this tool as a process, and cut out our creditors altogether!

## Tensorflow!

In [85]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.7.0


## Data!

Next, we'd like to get access to our training data-set. Ideally this should be sourced from Yoco itself, but for demo purposes, maybe not quite yet.

Instead we've merely preprocessed some data from the Yoco Core read replica. See `./data_transformations.ipynb`

In [109]:
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow.keras import layers

# First we load our collated transactional data
our_data = pd.read_csv('output_data/collated_output.csv')

# Next we need to take 80% of the records and call that out
# training set, and the remaining 20% will be our test set.
train, val, test = np.split(our_data.sample(frac=1), [int(0.8*len(our_data)), int(0.9*len(our_data))])

print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

# We'll also define some useful functions here now.

# This converts a pandas dataframe into a tf.data.Dataset
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
    df = dataframe.copy()
    labels = df.pop('capital_granted')
    df = {key: value[:,tf.newaxis] for key, value in dataframe.items()}
    ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))
    if shuffle:
      ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)
    ds = ds.prefetch(batch_size)
    return ds

# This function normalizes a numeric feature to the range [-1, 1]
def get_normalization_layer(name, dataset):
  # Create a Normalization layer for the feature.
  normalizer = layers.Normalization(axis=None)

  # Prepare a Dataset that only yields the feature.
  feature_ds = dataset.map(lambda x, y: x[name])

  # Learn the statistics of the data.
  normalizer.adapt(feature_ds)

  return normalizer

# This function converts a categorical feature to a multi-hot encoded tensor.
def get_category_encoding_layer(name, dataset, dtype, max_tokens=None):
      # Create a layer that turns strings into integer indices.
  if dtype == 'string':
    index = layers.StringLookup(max_tokens=max_tokens)
  # Otherwise, create a layer that turns integer values into integer indices.
  else:
    index = layers.IntegerLookup(max_tokens=max_tokens)

  # Prepare a `tf.data.Dataset` that only yields the feature.
  feature_ds = dataset.map(lambda x, y: x[name])

  # Learn the set of possible values and assign them a fixed integer index.
  index.adapt(feature_ds)

  # Encode the integer indices.
  encoder = layers.CategoryEncoding(num_tokens=index.vocabulary_size())

  # Apply multi-hot encoding to the indices. The lambda function captures the
  # layer, so you can use them, or include them in the Keras Functional model later.
  return lambda feature: encoder(index(feature))

2812 train examples
352 validation examples
352 test examples


In [115]:
%tensorboard --logdir logs/fit

UsageError: Line magic function `%tensorboard` not found.


In [114]:
import datetime

all_inputs = []
encoded_features = []

batch_size = 256
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

# Numerical features.
for header in ['accepted_txn', 'failed_txn', 'txn_vol']:
  numeric_col = tf.keras.Input(shape=(1,), name=header)
  normalization_layer = get_normalization_layer(header, train_ds)
  encoded_numeric_col = normalization_layer(numeric_col)
  all_inputs.append(numeric_col)
  encoded_features.append(encoded_numeric_col)
  
all_features = tf.keras.layers.concatenate(encoded_features)
x = tf.keras.layers.Dense(32, activation="relu")(all_features)
x = tf.keras.layers.Dropout(0.5)(x)
output = tf.keras.layers.Dense(1)(x)

model = tf.keras.Model(all_inputs, output)
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=["accuracy"])

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(train_ds, epochs=100, validation_data=val_ds, callbacks=[tensorboard_callback])

loss, accuracy = model.evaluate(test_ds)

/var/folders/_g/ch3mtw8j5m1_fvry4yt_gt_80000gn/T/ipykernel_4277/2429220327.py:24: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  df = {key: value[:,tf.newaxis] for key, value in dataframe.items()}


Epoch 1/100
11/11 [==============================] - 1s 23ms/step - loss: 0.7255 - accuracy: 0.1166 - val_loss: 0.6951 - val_accuracy: 0.0966
Epoch 2/100
11/11 [==============================] - 0s 9ms/step - loss: 0.6806 - accuracy: 0.1238 - val_loss: 0.6489 - val_accuracy: 0.0966
Epoch 3/100
11/11 [==============================] - 0s 9ms/step - loss: 0.6449 - accuracy: 0.1501 - val_loss: 0.6101 - val_accuracy: 0.0966
Epoch 4/100
11/11 [==============================] - 0s 8ms/step - loss: 0.6120 - accuracy: 0.2223 - val_loss: 0.5777 - val_accuracy: 0.0966
Epoch 5/100
11/11 [==============================] - 0s 10ms/step - loss: 0.5875 - accuracy: 0.3250 - val_loss: 0.5493 - val_accuracy: 0.4062
Epoch 6/100
11/11 [==============================] - 0s 8ms/step - loss: 0.5649 - accuracy: 0.4026 - val_loss: 0.5244 - val_accuracy: 0.5824
Epoch 7/100
11/11 [==============================] - 0s 7ms/step - loss: 0.5365 - accuracy: 0.4986 - val_loss: 0.5020 - val_accuracy: 0.6562
Epoch 8/100